In [108]:
# import dependencies and set starting alignment parameters
import pandas as pd
from Bio import pairwise2
gap_open_penalty = -1
gap_extend_penalty = -0.5

In [109]:
# set reference sequence to 602.2
refseq = 'MGESINFVSWGGSTQDAQKQAWADPFSKASGITVVQDGPTDYGKLKAMVESGNVQWDVVDVEADFALRAAAEGLLEPLDFSVIQRDKIDPRFVSDHGVGSFLFSFVLGYNEGKLGASKPQDWTALFDTKTYPGKRALYKWPSPGVLELALLADGVPADKLYPLDLDRAFKKLDTIKKDIVWWGGGAQSQQLLASGEVSMGQFWNGRIHALQEDGAPVGVSWKQNLVMADILVVPKGTKNKAAAMKFLASASSAKGQDDFSALTAYAPVNIDSVQRLDLAQVRITADKQKNGIMANFKIRHNVEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSVLSKDPNEKRDHMVLLEFVTAAGITLGMDELYKGGTGGSMSKGEELFTGVVPILVELDGDVNGHKFSVRGEGEGDATNGKLTLKFICTTGKLPVPWPTLVTTLTYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTISFKDDGTYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNWNANLAPNLPTAYVKDQITLDFAYWAKNGPAIATRWNEWLVKGSHHHHHH*'

In [110]:
# read csv of dna sequences and generate translations (?) not yet implemented

In [111]:
# read the csv of PROTEIN sequences
prot_seq = pd.read_csv('test.csv')

In [112]:
prot_seq

,Plate,Well,Sequence
0,M0025,A02,MGESINFVSWGGSTQDAQKQAWADPFSKASGITVVQDGPTDYGKLK...
1,M0026,A12,MGESINFVSWGGSTQDAQKQAWADPFSKASGITVVQDGPTDYGKLK...
2,M0026,D07,MGESINFVSWGGSTQDAQKQAWADPFSKASGITVVQDGPTDYGKLK...
3,M0027,E02,MGESINFVSWGGSTQDAQKQAWADPFSKASGITVVQDGPTDYGKLK...


In [113]:
prot_seq.shape
print(prot_seq.shape)

numItems = prot_seq.shape[0]
print(numItems)

(4, 3)
4


In [114]:
# Create a mutations column for all entries
prot_seq.loc[(prot_seq['Plate'] != ''), 'Mutations'] = ''
prot_seq

,Plate,Well,Sequence,Mutations
0,M0025,A02,MGESINFVSWGGSTQDAQKQAWADPFSKASGITVVQDGPTDYGKLK...,
1,M0026,A12,MGESINFVSWGGSTQDAQKQAWADPFSKASGITVVQDGPTDYGKLK...,
2,M0026,D07,MGESINFVSWGGSTQDAQKQAWADPFSKASGITVVQDGPTDYGKLK...,
3,M0027,E02,MGESINFVSWGGSTQDAQKQAWADPFSKASGITVVQDGPTDYGKLK...,


In [115]:
# derived from /tsanga/code/find_mutations.py
# version 2021-08-18

# go through the sequences and compare each to 602.2. 
# create a column in the pandas table that shows the amino acid changes
# it's important to note that the residue numbering is specific to this analysis, it must be converted
# to Jonny's system for interpretation. 

for i in range(numItems):
    mutList = []
    
    # on each iteration, assign the entry sequence to seq
    seq = prot_seq.iloc[i]['Sequence']
    
    
    # find the differences in letters
    for j in range(len(seq)):
        if seq[j] != refseq[j]:
            if seq[j] == '*':
                mutation = refseq[j] + str(j+1) + 'STOP'
            else:
                mutation = refseq[j] + str(j+1) + seq[i]
            mutList.append(mutation)
            
    # convert array to string
    mutString = str(mutList)
            
    # assign the mutation list to the correct item by matching the sequence
    prot_seq.loc[(prot_seq["Sequence"] == seq), 'Mutations'] = mutString
    


In [116]:
prot_seq

,Plate,Well,Sequence,Mutations
0,M0025,A02,MGESINFVSWGGSTQDAQKQAWADPFSKASGITVVQDGPTDYGKLK...,"['D60M', 'S100M', 'F103M', 'Y138M', 'K254M', '..."
1,M0026,A12,MGESINFVSWGGSTQDAQKQAWADPFSKASGITVVQDGPTDYGKLK...,"['D60G', 'S100G', 'F103G', 'A116G', 'K254G', '..."
2,M0026,D07,MGESINFVSWGGSTQDAQKQAWADPFSKASGITVVQDGPTDYGKLK...,"['D60E', 'S100E', 'F103E', 'K254E', 'L309E', '..."
3,M0027,E02,MGESINFVSWGGSTQDAQKQAWADPFSKASGITVVQDGPTDYGKLK...,"['S100S', 'F103S', 'T130S', 'M227S', 'K254S', ..."


In [117]:
prot_seq.to_csv('Analyzed_Sequences.csv')